In [51]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd 
from io import StringIO
from tabulate import tabulate

# 請幫我將上面的程式碼打包成一個function
def get_token_and_cookie():
    # 設定目標網址
    url = "https://www.tdcc.com.tw/portal/zh/smWeb/qryStock"  # 替換為你的目標網址

    # 初始化 WebDriver（例如使用 Chrome）
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # 如果不需要視覺化瀏覽器，加入這行
    driver = webdriver.Chrome(options=options)
    try:
        # 開啟網頁
        driver.get(url)
        
        # 等待網頁加載完成，直到指定元素出現（id 為 SYNCHRONIZER_TOKEN）
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "SYNCHRONIZER_TOKEN"))
        )
        
        # 取得name為JSESSIONID,_ga,_fbp,_ga_PZZ1MN4BZ8的cookie 的值，並使用name=value的格式串成字串
        cookies = driver.get_cookies()
        target_cookie = ";".join([f"{cookie['name']}={cookie['value']}" for cookie in cookies if cookie['name'] in ["JSESSIONID","_ga","_fbp","_ga_PZZ1MN4BZ8"]])

        if not target_cookie:
            print(f"Cookie not found.")
        
        # 取得 HTML 元素的值（id 為 SYNCHRONIZER_TOKEN）
        token_element = driver.find_element(By.ID, "SYNCHRONIZER_TOKEN")
        token_value = token_element.get_attribute("value")
    finally:
        # 關閉瀏覽器
        driver.quit()

    return token_value, target_cookie

# 呼叫function
token_value, target_cookie = get_token_and_cookie()

# 將token_value和target_cookie印出來
print(token_value)
print(target_cookie)

# 設定請求的URL
url = "https://www.tdcc.com.tw/portal/zh/smWeb/qryStock"

# 設定請求標頭
headers = {
    'Cookie': target_cookie,
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    'Content-Type': 'application/x-www-form-urlencoded',
    'Origin': 'https://www.tdcc.com.tw',
    'Referer': 'https://www.tdcc.com.tw/portal/zh/smWeb/qryStock',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"'
}

# 設定請求內容
payload = {
    'SYNCHRONIZER_TOKEN': token_value,
    'SYNCHRONIZER_URI': '/portal/zh/smWeb/qryStock',
    'method': 'submit',
    'firDate': '20250117',
    'scaDate': '20241227',
    'sqlMethod': 'StockNo',
    'stockNo': '1101',
    'stockName': ''
}

# 發送POST請求
import requests
response = requests.post(url, headers=headers, data=payload)

# 檢查回應狀態
if response.status_code == 200:
    print("請求成功")
    # 取得response的html裡面class為table的元素
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', class_='table')

    # 將 table 的 HTML 包裝成 StringIO 文件對象
    table_html = StringIO(str(table))

    df = pd.read_html(table_html)
    df = df[0]
    # remove the last row
    df = df[:-1]
    # hide the index
    print(tabulate(df, headers='keys', tablefmt='grid', showindex=False))

else:
    print(f"請求失敗，狀態碼：{response.status_code}")
    print(response.text)



99a20037-0be0-4269-9d88-f697d856932e
_fbp=fb.2.1737540918774.369033869753084;_ga=GA1.1.1492859174.1737540918;_ga_PZZ1MN4BZ8=GS1.1.1737540918.1.0.1737540918.0.0.0;JSESSIONID=Y_OutFt1nzCwoNnha9Wi4FDSgrOLIkpbOtgYgB6o.tdccap01p
請求成功
+------+-------------------+--------+---------------+------------------------+
|   序 | 持股/單位數分級   |   人數 |   股數/單位數 |   占集保庫存數比例 (%) |
+======+===================+========+===============+========================+
|    1 | 1-999             | 119567 |      22157943 |                   0.29 |
+------+-------------------+--------+---------------+------------------------+
|    2 | 1,000-5,000       | 251558 |     554860685 |                   7.34 |
+------+-------------------+--------+---------------+------------------------+
|    3 | 5,001-10,000      |  62278 |     454615066 |                   6.02 |
+------+-------------------+--------+---------------+------------------------+
|    4 | 10,001-15,000     |  25605 |     313487021 |                   4.15 |
+---